In [1]:
cd ..

/Users/m.dee/Documents/ML-Navigator


<img src="flow_0.png">

In [2]:
import os
from flows.flows import Flows

Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)
You can define the `categorical_threshold` which is the maximum number of categories that a categorical feature should have before considering it as continuous numeric feature. The default value is 50
For example, use the code below to import the flow 0 with defining the categorical_threshold as 50
flow = Flows(flow_id=0, categorical_threshold=50)


In [13]:
flow = Flows(flow_id=0, categorical_threshold=10)

Please use the following function to read the data
dataframe_dict = flow.load_data(path: str, files_list: list)
For example: path = './data'
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.  
dataframe_dict = {'train': train_dataframe,'test': test_dataframe}


In [14]:
path = os.path.join('data','flow_0')
files_list = ["train.csv","test.csv"]

In [15]:
dataframe_dict, columns_set = flow.load_data(path, files_list)

A summary of the data sets


,train,test
column type,,
categorical_integer,12,12
categorical_string,43,43
continuous,26,25
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 10 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
The possible ids are {'Id'}
The possible possible_target are ['SalePrice']
The type of the problem that should be solved {'SalePrice': 'regression'}
If you have categorical features with string labels, Encode the categorical features by applying the following function:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict: dict)


In [16]:
 columns_set["train"].keys()

dict_keys(['categorical_string', 'categorical_integer', 'continuous', 'date', 'json', 'other'])

In [17]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict)

The reference dataframe is: train
******************************
A summary of the data sets


,train,test
column type,,
categorical_integer,52,52
categorical_string,0,0
continuous,29,28
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 10 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If you have numeric features, it is a good idea to normalize numeric features. Use the following function for feature normalization :
dataframe_dict, columns_set = flow.scale_data (dataframe_dict: dict, ignore_columns: list)
For example: ignore_columns = ['id', 'target']


In [18]:
ignore_columns = ['Id', 'SalePrice']

In [19]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

A summary of the data sets


,train,test
column type,,
categorical_integer,52,52
categorical_string,0,0
continuous,29,28
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 10 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
Your features are ready to train the model: 
If you want to explore the data you can run one of the following functions: 
1 . flow.exploring_data(dataframe_dict: dict, key_i: str)
For example: flow.exploring_data(dataframe_dict, 'train')
2 . flow.comparing_statistics(dataframe_dict: dict)
For example: flow.comparing_statistics(dataframe_dict)




You can start training the model by applying the following function: 
model_index_list, save_models_dir, y_test = flow.training(parameters)
parameters = { 
 "data": {
 "train": {"features": train_dataframe, "target": train_target},
 "valid": {"features": valid_dataframe, "target": valid_target},
 "test": {"features": test_dataframe, "target": test_target},
 },
 "split": {
 "method": "split",  # "

In [20]:
flow.exploring_data(dataframe_dict, "test")

interactive(children=(IntSlider(value=0, description='column_i', max=79), Output()), _dom_classes=('widget-int…

In [21]:
flow.comparing_statistics(dataframe_dict)

interactive(children=(IntSlider(value=0, description='feature_nr', max=80), Output()), _dom_classes=('widget-i…

In [22]:
ignore_columns = ["Id", "SalePrice"]
columns = dataframe_dict["train"].columns
train_dataframe = dataframe_dict["train"][[x for x in columns_set["train"]["continuous"] if x not in ignore_columns]]
test_dataframe = dataframe_dict["test"][[x for x in columns_set["train"]["continuous"] if x not in ignore_columns]]
train_target = dataframe_dict["train"]["SalePrice"]

In [23]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "Ridge linear regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["r2_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [24]:
model_index_list, save_models_dir, y_test = flow.training(parameters)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
the required metrics =  ['r2_score']
The quality of the model using the dataset kfold 1
dataset kfold 1:  R2: 78.5 %
fold_nr. 2
the required metrics =  ['r2_score']
The quality of the model using the dataset kfold 2
dataset kfold 2:  R2: 55.4 %
fold_nr. 3
the required metrics =  ['r2_score']
The quality of the model using the dataset kfold 3
dataset kfold 3:  R2: 76.8 %
fold_nr. 4
the required metrics =  ['r2_score']
The quality of the model using the dataset kfold 4
dataset kfold 4:  R2: 70.2 %
fold_nr. 5
the required metrics =  ['r2_score']
The quality of the model using the dataset kfold 5
dataset kfold 5:  R2: 79.5 %


,fold_1,fold_2,fold_3,fold_4,fold_5,mean
r2_score,78.5,55.4,76.8,70.2,79.5,72.08


the required metrics =  ['r2_score']
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train:  R2: 75.1 %
the required metrics =  ['r2_score']
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train:  R2: 74.4 %
the required metrics =  ['r2_score']
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train:  R2: 75.3 %
the required metrics =  ['r2_score']
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train:  R2: 75.4 %
the required metrics =  ['r2_score']
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train:  R2: 75.3 %
train:  Mean R2: 75.10000000000001 %


,model_1,model_2,model_3,model_4,model_5,mean
r2_score (train),75.1,74.4,75.3,75.4,75.3,75.1


This is the end of the flow


In [ ]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='regression', metric='root_mean_squared_error', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters_lighgbm)